In [ ]:
## Code for the publication 

## Validating and Utilizing Machine Learning Methods to Investigate 
## the Impacts of Synthesis Parameters in Gold Nanoparticle Synthesis"
## by Daniel Schletz, Morten Breidung, and Andreas Fery

## CC BY-NC-ND 4.0

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MaxAbsScaler
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
#Import Parameter+DLS
a = pd.read_excel("Data/Synthesis_Parameters_DLS.xlsx")
#Import Spectra
b = pd.read_excel("Data/UV-vis_Spectra.xlsx")
#Import Abs_Max Data
c = pd.read_excel("Data/Absorption_Maximum_FW80M.xlsx")

In [ ]:
#Prepare Parameters
X = a.drop(columns=["Name","DLS Diameter [nm]","Standard Deviation DLS [nm]"])

In [ ]:
#### Begin Data Collection

In [ ]:
def NestedCV_LR(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
        # define the model
        model = LinearRegression()
        # define search
        space = dict()
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    # save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()
    
    return y_graph_test, y_graph_hat

In [ ]:
def NestedCV_RF(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
        # define the model
        model = RandomForestRegressor(random_state=42, n_jobs = -1)
        # define search space
        space = dict()
        space['n_estimators'] = [500, 700, 900, 1000, 1100, 1300, 1500]
        space['max_features'] = [1.0, 'sqrt', 0.3]
        space['min_samples_leaf'] = [1, 2, 4]
        space['min_samples_split'] = [2, 4, 8, 16]
        # define search
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    # save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()
    
    return y_graph_test, y_graph_hat

In [ ]:
def NestedCV_AdaB(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
        # define the model
        model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(),random_state=42)
        # define search space
        space = dict()
        space['n_estimators'] = [300, 500]#, 700, 900, 1000, 1100, 1300, 1500]
        space['base_estimator__max_depth'] = [None, 6, 8, 10, 12] 
        space['learning_rate'] = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
        # define search
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        #print('>acc[MSE]=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    # save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()

    return y_graph_test, y_graph_hat

In [ ]:
def NestedCV_XGB(XData, yData, savetest, savehat):
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
    # enumerate splits
    outer_results = list()
    outer_results2 = list()
    y_graph_hat = list()
    y_graph_test = list()
    X2 = XData.to_numpy()
    y2 = yData.to_numpy().ravel()
    for train_ix, test_ix in cv_outer.split(X):
    # split data
        X_train, X_test = X2[train_ix, :], X2[test_ix, :]
        y_train, y_test = y2[train_ix], y2[test_ix]
        y_graph_test.append(y_test)
        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
        # define the model
        model = XGBRegressor(random_state=42, tree_method = 'exact', n_jobs = -1)
        # define search space
        space = dict()
        space['learning_rate'] = [0.1, 0.2, 0.3, 0.4, 0.5]
        space['max_depth']= [6]#None, 6]
        space['colsample_bytree'] = [0.5,1.0]
        space['colsample_bylevel'] = [0.5,1.0]
        # define search
        search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
        # execute search
        result = search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        # evaluate model on the hold out dataset
        yhat = best_model.predict(X_test)
        y_graph_hat.append(yhat)
        # evaluate the model
        acc = mean_squared_error(y_test, yhat)
        acc2 = r2_score(y_test, yhat)
        # store the result
        outer_results.append(acc)
        outer_results2.append(acc2)
        # report progress
        print('>acc[MSE]=%.3f, acc=%.3f, est=%.3f, cfg=%s' % (acc, acc2, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
    #save data
    f = open(savetest, 'wb')
    pickle.dump(y_graph_test, f)
    f.close()
    f = open(savehat, 'wb')
    pickle.dump(y_graph_hat, f)
    f.close()

    return y_graph_test, y_graph_hat

In [ ]:
#Prepare Abs_Max Target
y = c.drop(columns=["Name", "Abs max", "FW80M"])
test_MaxAbs_LR, hat_MaxAbs_LR = NestedCV_LR(X,y, "FW_LR_test.pckl", "FW_LR_hat.pckl")
test_MaxAbs_RF, hat_MaxAbs_RF = NestedCV_RF(X,y, "MaxAbs_RF_test.pckl", "MaxAbs_RF_hat.pckl")
test_MaxAbs_AdaB, hat_MaxAbs_AdaB = NestedCV_AdaB(X,y, "MaxAbs_AdaB_test.pckl", "MaxAbs_AdaB_hat.pckl")
test_MaxAbs_XGB, hat_MaxAbs_XGB = NestedCV_XGB(X,y, "MaxAbs_XGB_test.pckl", "MaxAbs_XGB_hat.pckl")

In [ ]:
#Prepare FW80M Target
y = c.drop(columns=["Name", "Abs max", "Wavelength [nm]"])
test_FW_LR, hat_FW_LR = NestedCV_LR(X,y, "FW_LR_test.pckl", "FW_LR_hat.pckl")
test_FW_RF, hat_FW_RF = NestedCV_RF(X,y, "FW_RF_test.pckl", "FW_RF_hat.pckl")
test_FW_AdaB, hat_FW_AdaB = NestedCV_AdaB(X,y, "FW_AdaB_test.pckl", "FW_AdaB_hat.pckl")
test_FW_XGB, hat_FW_XGB = NestedCV_XGB(X,y, "FW_XGB_test.pckl", "FW_XGB_hat.pckl")


In [ ]:
#Prepare DLS Target
y = a["DLS Diameter [nm]"]
test_Dia_LR, hat_Dia_LR = NestedCV_LR(X,y, "Dia_LR_test.pckl", "Dia_LR_hat.pckl")
test_Dia_RF, hat_Dia_RF = NestedCV_RF(X,y, "Dia_RF_test.pckl", "Dia_RF_hat.pckl")
test_Dia_AdaB, hat_Dia_AdaB = NestedCV_AdaB(X,y, "Dia_AdaB_test.pckl", "Dia_AdaB_hat.pckl")
test_Dia_XGB, hat_Dia_XGB = NestedCV_XGB(X,y, "Dia_XGB_test.pckl", "Dia_XGB_hat.pckl")


In [ ]:
## Random Forest Full Spectrum
# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
outer_results2 = list()
y_graph_hat_CombRF = list()
y_graph_test_CombRF = list()
#Normalize Spectra
scaler=MaxAbsScaler()
b = b.drop(columns=["Wavelength [nm]"])
y = pd.DataFrame(scaler.fit_transform(b.T).T,columns=b.columns)
X2 = X.to_numpy()
y2 = y.to_numpy()
for train_ix, test_ix in cv_outer.split(X):
# split data
    X_train, X_test = X2[train_ix, :], X2[test_ix, :]
    y_train, y_test = y2[train_ix], y2[test_ix]
    y_graph_test_CombRF.append(y_test)
    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
    # define the model
    model = RandomForestRegressor(random_state=42, n_jobs = -1)
    # define search space
    space = dict()
    space['n_estimators'] = [500, 700, 900, 1000, 1100, 1300, 1500]
    space['max_features'] = [1.0, 'sqrt', 0.3]
    space['min_samples_leaf'] = [1, 2, 4]
    space['min_samples_split'] = [2, 4, 8, 16]
    # define search
    search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    y_graph_hat_CombRF.append(yhat)
    # evaluate the model
    acc = mean_squared_error(y_test, yhat)
    acc2 = r2_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    outer_results2.append(acc2)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc2, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Accuracy: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))

In [ ]:
##Final model training for Random Forest Abs_Max for SHAP Analysis
y = c.drop(columns=["Name", "Abs max", "FW80M"])
cv_inner = KFold(n_splits=10, shuffle=True, random_state=1)
# define the model
model = RandomForestRegressor(random_state=42, n_jobs = -1)
space = dict()
space['n_estimators'] = [500, 700, 900, 1000, 1100, 1300, 1500]
space['max_features'] = [1.0, 'sqrt', 0.3]
space['min_samples_leaf'] = [1, 2, 4]
space['min_samples_split'] = [2, 4, 8, 16]
# define search
search = GridSearchCV(model, space, scoring='r2', cv=cv_inner, refit=True, n_jobs=-1)
# execute search
result = search.fit(X.to_numpy(), y.to_numpy().ravel())
print('>acc=%.3f, cfg=%s' % (result.best_score_, result.best_params_))


In [ ]:
model = search.best_estimator_
y = c.drop(columns=["Name", "Abs max", "FW80M"])
model.fit(X, y.to_numpy().ravel())